In [ ]:
pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.metrics import mean_absolute_error
import scipy.stats
import numpy as np
import statsmodels.api as sm

# 使用allRNA,直接使用整个数据集进行训练

In [ ]:
# 读取数据
mirna = pd.read_csv('./all_ori_new.csv',index_col=0)
# 删除空值行
mirna = mirna.dropna(subset=['HAMD'])

# 提取特征和标签
X = mirna.drop(columns=['HAMD'])
y = mirna['HAMD']
y


In [ ]:
mirna

In [ ]:
X

In [ ]:

import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold

# 设置 l1_ratio 和 alpha 的范围
ratios = np.arange(0.01, 1.01, 0.05)
alphas = np.concatenate((
    [10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 0],
    np.arange(0.1, 1.0, 0.1),
    np.arange(1, 11, 1)
))

# 设置交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 记录总的模型数量
total_models = len(ratios) * len(alphas) * kf.get_n_splits(X)
progress = 0

# 初始化变量以记录最佳参数和误差
best_ratio = None
best_alpha = None
best_mse = float('inf')
best_r2 = float('-inf')

# 打开文件记录训练进度和结果
with open('elastic_net_cv_progress.txt', 'w') as f:

    # 开始手动交叉验证
    for ratio in ratios:
        for alpha in alphas:
            for fold, (train_index, val_index) in enumerate(kf.split(X)):
                X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
                y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

                # 创建和训练 ElasticNet 模型
                model = ElasticNet(alpha=alpha, l1_ratio=ratio, random_state=42, max_iter=10000)
                model.fit(X_train_fold, y_train_fold)

                # 预测和计算 MSE 和 R2
                y_val_pred = model.predict(X_val_fold)
                mse = mean_squared_error(y_val_fold, y_val_pred)
                r2 = r2_score(y_val_fold, y_val_pred)

                # 记录最佳参数和误差
                if mse < best_mse:
                    best_mse = mse
                    best_ratio = ratio
                    best_alpha = alpha
                    best_r2 = r2

                # 更新进度
                progress += 1
                progress_msg = (f"训练进度: 第 {progress} / {total_models} 个模型 - "
                                f"当前 l1_ratio: {ratio}, alpha: {alpha}, MSE: {mse}, R2: {r2}")
                print(progress_msg)
                f.write(progress_msg + '\n')

    # 输出最终结果
    result_msg = (f"最佳平均交叉验证误差(MSE): {best_mse}\n"
                  f"最佳 l1_ratio: {best_ratio}\n"
                  f"最佳 alpha: {best_alpha}\n"
                  f"最佳 R2: {best_r2}\n")
    print(result_msg)
    f.write(result_msg)


In [ ]:
print(best_ratio)
print(best_alpha)
print(best_r2)
print(best_mse)

In [ ]:
result_msg = (f"最佳平均交叉验证误差(MSE): {best_mse}\n"
                  f"最佳 l1_ratio: {best_ratio}\n"
                  f"最佳 alpha: {best_alpha}\n"
                  f"最佳 R2: {best_r2}\n")
print(result_msg)


In [ ]:
import concurrent.futures

X_array = X.to_numpy()
y_array = y.to_numpy()

num_bootstrap = 100
selected_features = np.zeros((X.shape[1], num_bootstrap))

# 定义一个函数，用于每个 Bootstrap 迭代
def bootstrap_iteration(i):
    bootstrap_indices = np.random.choice(range(X_array.shape[0]), size=int(0.8 * X_array.shape[0]), replace=True)
    X_bootstrap = X_array[bootstrap_indices]
    y_bootstrap = y_array[bootstrap_indices]
    
    # 使用 Elastic Net 模型拟合数据
    elastic_net = ElasticNet(alpha=best_alpha, l1_ratio=best_ratio, random_state=42)
    elastic_net.fit(X_bootstrap, y_bootstrap)
    
    return elastic_net.coef_ != 0

# 使用多线程并行化 Bootstrap 循环
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(bootstrap_iteration, range(num_bootstrap)))

selected_features = np.array(results).T

# 计算每个特征被选择的频率
feature_selection_frequency = np.mean(selected_features, axis=1)

# 创建 DataFrame
df_feature_selection = pd.DataFrame({
    'Feature': range(1, X.shape[1] + 1),
    'Selection Frequency': feature_selection_frequency
})

# 将结果保存为 CSV 文件
csv_file_path = './feature_selection_frequency.csv'
df_feature_selection.to_csv(csv_file_path, index=False)

print(f"特征选择频率已保存到 {csv_file_path}")

In [ ]:
feature_selection_frequency

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# 设置 feature_selection_frequency 阈值范围
thresholds = np.arange(1.0, 0.0, -0.01)  # 从 1 到 0.1，步长 0.1

best_threshold = None
best_r2 = float('-inf')  # 设定最初的最佳 R² 为负无穷

# 循环遍历不同的 feature_selection_frequency 阈值
for threshold in thresholds:
    # 选择 feature_selection_frequency 大于等于当前阈值的特征
    selected_feature_mask = feature_selection_frequency >= threshold
    X_selected = X.iloc[:, selected_feature_mask]
    
    # 如果没有任何特征被选中，跳过当前阈值
    if X_selected.shape[1] == 0:
        print(f"Warning: No features selected at threshold {threshold}. Skipping...")
        continue
    
    mse_list = []
    r2_list = []
    
    # 进行 101 次 80% / 20% 划分
    for i in range(101):
        # 随机划分数据集为 80% 训练集和 20% 验证集
        X_train, X_val, y_train, y_val = train_test_split(X_selected, y, train_size=0.8, random_state=42)

        # 使用 ElasticNet 模型进行训练
        elastic_net = ElasticNet(alpha=best_alpha, l1_ratio=best_ratio, random_state=42)
        elastic_net.fit(X_train, y_train)
        
        # 在验证集上预测
        y_pred_val = elastic_net.predict(X_val)
        
        # 计算 MSE 和 R²
        mse = mean_squared_error(y_val, y_pred_val)
        r2 = r2_score(y_val, y_pred_val)
        
        mse_list.append(mse)
        r2_list.append(r2)

    # 计算该阈值下的平均 MSE 和 R²
    avg_mse = np.mean(mse_list)
    avg_r2 = np.mean(r2_list)

    # 如果当前阈值下的 R² 更高，更新最佳阈值和 R²
    if avg_r2 > best_r2:
        best_r2 = avg_r2
        best_threshold = threshold

    print(f"Threshold: {threshold}, Avg R²: {avg_r2}, Avg MSE: {avg_mse}")

# 输出最佳的 feature_selection_frequency 阈值和对应的平均 R²
print(f"\n最佳 feature_selection_frequency 阈值: {best_threshold}")
print(f"最佳平均 R²: {best_r2}")


In [ ]:
best_threshold = 0.97
best_threshold

In [ ]:
# 选择 feature_selection_frequency 的特征进行 10 次交叉验证，并计算均方误差 (MSE) 和 R² 等评估指标
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold

# 设置 feature_selection_frequency 阈值范围
thresholds = np.arange(1.0, 0.0, -0.1)  # 从 1 到 0.1，步长 0.1

best_threshold = None
best_r2 = float('-inf')  # 设定最初的最佳 R² 为负无穷

# 循环遍历不同的 feature_selection_frequency 阈值
for threshold in thresholds:
    # 选择 feature_selection_frequency 大于等于当前阈值的特征
    selected_feature_mask = feature_selection_frequency >= threshold
    X_selected = X.iloc[:, selected_feature_mask]

    # 如果没有任何特征被选中，跳过当前阈值
    if X_selected.shape[1] == 0:
        print(f"Warning: No features selected at threshold {threshold}. Skipping...")
        continue
    
    # 10 折交叉验证
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    mse_list = []
    r2_list = []

    for train_index, val_index in kf.split(X_selected):
        X_train_fold, X_val_fold = X_selected.iloc[train_index], X_selected.iloc[val_index]
        y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
        
        # 使用 ElasticNet 模型进行训练
        elastic_net = ElasticNet(alpha=best_alpha, l1_ratio=best_ratio, random_state=42)
        elastic_net.fit(X_train_fold, y_train_fold)
        
        # 在验证集上预测
        y_pred_val = elastic_net.predict(X_val_fold)
        
        # 计算 MSE 和 R²
        mse = mean_squared_error(y_val_fold, y_pred_val)
        r2 = r2_score(y_val_fold, y_pred_val)
        
        mse_list.append(mse)
        r2_list.append(r2)

    # 计算该阈值下的平均 MSE 和 R²
    avg_mse = np.mean(mse_list)
    avg_r2 = np.mean(r2_list)

    # 如果当前阈值下的 R² 更高，更新最佳阈值和 R²
    if avg_r2 > best_r2:
        best_r2 = avg_r2
        best_threshold = threshold

    print(f"Threshold: {threshold}, Avg R²: {avg_r2}, Avg MSE: {avg_mse}")

# 输出最佳的 feature_selection_frequency 阈值和对应的平均 R²
print(f"\n最佳 feature_selection_frequency 阈值: {best_threshold}")
print(f"最佳平均 R²: {best_r2}")

# # 保存模型
# model_file_path = 'D:/adult_dep/12.28HAMD拟合/all_model.joblib'
# joblib.dump(elastic_net, model_file_path)
# print(f"模型已保存到 {model_file_path}")

# # 在整个数据集上训练最终的模型
# final_model = ElasticNet(alpha=6.0, l1_ratio=0.01, random_state=42)
# final_model.fit(X_selected, y)

# # 在训练集上进行预测
# y_pred_train = final_model.predict(X_selected)

# # 计算训练集上的评估指标
# train_mse = mean_squared_error(y, y_pred_train)
# train_r2 = r2_score(y, y_pred_train)

# print(f"训练集上的 MSE: {train_mse}")
# print(f"训练集上的 R²: {train_r2}")

# 0.7，以此为准
# 平均 MSE: 54.84048846110075
# 平均 R²: 0.6495596148017435
# 训练集上的 MSE: 25.633336819521226
# 训练集上的 R²: 0.8375057719150996

# 0.8
# 平均 MSE: 53.75572059128236
# 平均 R²: 0.6564829573502353
# 训练集上的 MSE: 25.88240394821413
# 训练集上的 R²: 0.8359268916037601

In [ ]:
best_alpha

In [ ]:
best_ratio

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold
from scipy.stats import pearsonr

# 假设已经计算并找到了最佳的 feature_selection_frequency 阈值
best_threshold = best_threshold

# 选择 feature_selection_frequency 大于等于最佳阈值的特征
selected_feature_mask = feature_selection_frequency >= best_threshold
X_selected = X.iloc[:, selected_feature_mask]

# 10 折交叉验证
kf = KFold(n_splits=10, shuffle=True, random_state=42)
mse_list = []
r2_list = []
mae_list = []
pearson_corr_list = []

for train_index, val_index in kf.split(X_selected):
    X_train_fold, X_val_fold = X_selected.iloc[train_index], X_selected.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
    # 使用 ElasticNet 模型进行训练
    elastic_net = ElasticNet(alpha=best_alpha, l1_ratio=best_ratio, random_state=42)
    elastic_net.fit(X_train_fold, y_train_fold)
    
    # 在验证集上预测
    y_pred_val = elastic_net.predict(X_val_fold)
    
    # 计算 MSE 和 R²
    mse = mean_squared_error(y_val_fold, y_pred_val)
    r2 = r2_score(y_val_fold, y_pred_val)
    mae = mean_absolute_error(y_val_fold, y_pred_val)
    pearson_corr, _ = pearsonr(y_val_fold, y_pred_val)
    
    # 将每次折叠的结果添加到列表中
    mse_list.append(mse)
    r2_list.append(r2)
    mae_list.append(mae)
    pearson_corr_list.append(pearson_corr)

# 计算 10 折交叉验证的平均值
avg_mse = np.mean(mse_list)
avg_r2 = np.mean(r2_list)
avg_mae = np.mean(mae_list)
avg_pearson_corr = np.mean(pearson_corr_list)

# 输出最终的评估指标
print(f"\n基于最佳阈值 {best_threshold} 的评估结果：")
print(f"平均 R²: {avg_r2}")
print(f"平均 MAE: {avg_mae}")
print(f"平均 Pearson 相关系数: {avg_pearson_corr}")


In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

# 假设已经计算并找到了最佳的 feature_selection_frequency 阈值
best_threshold = best_threshold

# 选择 feature_selection_frequency 大于等于最佳阈值的特征
selected_feature_mask = feature_selection_frequency >= best_threshold
X_selected = X.iloc[:, selected_feature_mask]

# 10 折交叉验证
kf = KFold(n_splits=10, shuffle=True, random_state=42)
mse_list = []
r2_list = []
mae_list = []
pearson_corr_list = []
y_true_all = []  # 用于存储所有的真实值
y_pred_all = []  # 用于存储所有的预测值

for train_index, val_index in kf.split(X_selected):
    X_train_fold, X_val_fold = X_selected.iloc[train_index], X_selected.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
    # 使用 ElasticNet 模型进行训练
    elastic_net = ElasticNet(alpha=best_alpha, l1_ratio=best_ratio, random_state=42)
    elastic_net.fit(X_train_fold, y_train_fold)
    
    # 在验证集上预测
    y_pred_val = elastic_net.predict(X_val_fold)
    
    # 计算 MSE 和 R²
    mse = mean_squared_error(y_val_fold, y_pred_val)
    r2 = r2_score(y_val_fold, y_pred_val)
    mae = mean_absolute_error(y_val_fold, y_pred_val)
    pearson_corr, _ = pearsonr(y_val_fold, y_pred_val)
    
    # 将每次折叠的结果添加到列表中
    mse_list.append(mse)
    r2_list.append(r2)
    mae_list.append(mae)
    pearson_corr_list.append(pearson_corr)
    
    # 存储当前折叠的真实值和预测值
    y_true_all.extend(y_val_fold)
    y_pred_all.extend(y_pred_val)

# 计算 10 折交叉验证的平均值
avg_mse = np.mean(mse_list)
avg_r2 = np.mean(r2_list)
avg_mae = np.mean(mae_list)
avg_pearson_corr = np.mean(pearson_corr_list)

# 输出最终的评估指标
print(f"\n基于最佳阈值 {best_threshold} 的评估结果：")
print(f"平均 R²: {avg_r2}")
print(f"平均 MAE: {avg_mae}")
print(f"平均 Pearson 相关系数: {avg_pearson_corr}")

# 绘制散点图
plt.figure(figsize=(8, 8))  # 设置图形为正方形
sns.scatterplot(x=y_true_all, y=y_pred_all, alpha=0.6, edgecolor=None)
sns.regplot(x=y_true_all, y=y_pred_all, scatter=False, color='black')

# 设置坐标轴范围
plt.xlim(min(y_true_all) - 5, max(y_true_all) + 10)
plt.ylim(min(y_pred_all) - 5, max(y_pred_all) + 10)

# 设置坐标轴标签和标题
plt.xlabel('true')
plt.ylabel('predicted')
plt.title('Correlation')

# 保存图形为 PDF 文件
# pdf_file_path = 'D:/代谢组/data/真实值与预测值散点图.pdf'
# plt.savefig(pdf_file_path, format='pdf')

# 显示图形
plt.show()


In [ ]:
pip install YlGnOr

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

# 假设已经计算并找到了最佳的 feature_selection_frequency 阈值
best_threshold = best_threshold

# 选择 feature_selection_frequency 大于等于最佳阈值的特征
selected_feature_mask = feature_selection_frequency >= best_threshold
X_selected = X.iloc[:, selected_feature_mask]

# 10 折交叉验证
kf = KFold(n_splits=10, shuffle=True, random_state=42)
mse_list = []
r2_list = []
mae_list = []
pearson_corr_list = []
y_true_all = []  # 用于存储所有的真实值
y_pred_all = []  # 用于存储所有的预测值

for train_index, val_index in kf.split(X_selected):
    X_train_fold, X_val_fold = X_selected.iloc[train_index], X_selected.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
    # 使用 ElasticNet 模型进行训练
    elastic_net = ElasticNet(alpha=best_alpha, l1_ratio=best_ratio, random_state=42)
    elastic_net.fit(X_train_fold, y_train_fold)
    
    # 在验证集上预测
    y_pred_val = elastic_net.predict(X_val_fold)
    
    # 计算 MSE 和 R²
    mse = mean_squared_error(y_val_fold, y_pred_val)
    r2 = r2_score(y_val_fold, y_pred_val)
    mae = mean_absolute_error(y_val_fold, y_pred_val)
    pearson_corr, _ = pearsonr(y_val_fold, y_pred_val)
    
    # 将每次折叠的结果添加到列表中
    mse_list.append(mse)
    r2_list.append(r2)
    mae_list.append(mae)
    pearson_corr_list.append(pearson_corr)
    
    # 存储当前折叠的真实值和预测值
    y_true_all.extend(y_val_fold)
    y_pred_all.extend(y_pred_val)

# 计算 10 折交叉验证的平均值
avg_mse = np.mean(mse_list)
avg_r2 = np.mean(r2_list)
avg_mae = np.mean(mae_list)
avg_pearson_corr = np.mean(pearson_corr_list)

# 输出最终的评估指标
print(f"\n基于最佳阈值 {best_threshold} 的评估结果：")
print(f"平均 R²: {avg_r2}")
print(f"平均 MAE: {avg_mae}")
print(f"平均 Pearson 相关系数: {avg_pearson_corr}")

# 将真实值和预测值转为DataFrame
df_predictions = pd.DataFrame({'True': y_true_all, 'Pred': y_pred_all})

# 绘制箱型图
plt.figure(figsize=(8, 6))
sns.regplot(x=y_true_all, y=y_pred_all, scatter=False, color='black')
# 使用seaborn的boxplot绘制箱型图
sns.boxplot(x='True', y='Pred', data=df_predictions)

# 设置坐标轴标签和标题
plt.xlabel('True HAMD score')
plt.ylabel('Predicted HAMD score')
plt.title('Correlation')

# 显示图形
plt.show()


In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import matplotlib

# 假设已经计算并找到了最佳的 feature_selection_frequency 阈值
best_threshold = best_threshold

# 选择 feature_selection_frequency 大于等于最佳阈值的特征
selected_feature_mask = feature_selection_frequency >= best_threshold
X_selected = X.iloc[:, selected_feature_mask]

# 10 折交叉验证
kf = KFold(n_splits=10, shuffle=True, random_state=42)
mse_list = []
r2_list = []
mae_list = []
pearson_corr_list = []
y_true_all = []  # 用于存储所有的真实值
y_pred_all = []  # 用于存储所有的预测值

for train_index, val_index in kf.split(X_selected):
    X_train_fold, X_val_fold = X_selected.iloc[train_index], X_selected.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
    # 使用 ElasticNet 模型进行训练
    elastic_net = ElasticNet(alpha=best_alpha, l1_ratio=best_ratio, random_state=42)
    elastic_net.fit(X_train_fold, y_train_fold)
    
    # 在验证集上预测
    y_pred_val = elastic_net.predict(X_val_fold)
    
    # 计算 MSE 和 R²
    mse = mean_squared_error(y_val_fold, y_pred_val)
    r2 = r2_score(y_val_fold, y_pred_val)
    mae = mean_absolute_error(y_val_fold, y_pred_val)
    pearson_corr, _ = pearsonr(y_val_fold, y_pred_val)
    
    # 将每次折叠的结果添加到列表中
    mse_list.append(mse)
    r2_list.append(r2)
    mae_list.append(mae)
    pearson_corr_list.append(pearson_corr)
    
    # 存储当前折叠的真实值和预测值
    y_true_all.extend(y_val_fold)
    y_pred_all.extend(y_pred_val)

# 计算 10 折交叉验证的平均值
avg_mse = np.mean(mse_list)
avg_r2 = np.mean(r2_list)
avg_mae = np.mean(mae_list)
avg_pearson_corr = np.mean(pearson_corr_list)

# 输出最终的评估指标
print(f"\n基于最佳阈值 {best_threshold} 的评估结果：")
print(f"平均 R²: {avg_r2}")
print(f"平均 MAE: {avg_mae}")
print(f"平均 Pearson 相关系数: {avg_pearson_corr}")

# 将真实值和预测值转为DataFrame
df_predictions = pd.DataFrame({'True': y_true_all, 'Pred': y_pred_all})

# 创建自定义颜色映射：从草绿色到暗橙色
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    "grass_to_orange", ["#8FBC8F", "#8FBC8F"], N=256
)

norm = Normalize(vmin=min(y_true_all), vmax=max(y_true_all))  # 规范化真实值

# 获取颜色
sm = ScalarMappable(cmap=cmap, norm=norm)
df_predictions['Color'] = df_predictions['True'].apply(lambda x: sm.to_rgba(x))

# 绘制箱型图
plt.figure(figsize=(8, 6))
sns.regplot(x=y_true_all, y=y_pred_all, scatter=False, color='grey')

# 使用seaborn的boxplot绘制箱型图，并设置颜色
sns.boxplot(x='True', y='Pred', data=df_predictions, palette=df_predictions['Color'])

# 手动设置 x 轴刻度：从 -3 到最大值，步长为 5
x_ticks = np.arange(-1, 36, 5)

# 设置 x 轴的范围（稍微增加两边的空白）
plt.xlim(-3, 39)

# 设置 x 轴的刻度和标签
plt.xticks(x_ticks)

# 设置坐标轴标签和标题
plt.xlabel('True HAMD score')
plt.ylabel('Predicted HAMD score')
plt.title('Correlation')

# 添加评估指标文本（R², MAE, Pearson 相关系数）
plt.text(0.95, 0.25, f'R²: {avg_r2:.3f}', ha='right', va='top', transform=plt.gca().transAxes, fontsize=12, color='black')
plt.text(0.95, 0.20, f'MAE: {avg_mae:.3f}', ha='right', va='top', transform=plt.gca().transAxes, fontsize=12, color='black')
plt.text(0.95, 0.15, f'Pearson corr: {avg_pearson_corr:.3f}', ha='right', va='top', transform=plt.gca().transAxes, fontsize=12, color='black')

# 显示图形
pdf_file_path = './all_correlation_distribution.pdf'
plt.savefig(pdf_file_path, format='pdf')
plt.show()


In [ ]:
# 先不用管
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold

# 假设 X, y, 和 feature_selection_frequency 已经定义并准备好
# 选择 feature_selection_frequency >= 0.9 的特征
selected_feature_mask = feature_selection_frequency >= 0.9
X_selected = X.iloc[:, selected_feature_mask]

# 设置 10 折交叉验证
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 设置 ElasticNet 模型的超参数
best_alpha = 6.0  # 假设你已经选择了最优的 alpha 参数
best_l1_ratio = 0.01  # 假设你已经选择了最优的 l1_ratio 参数

# 初始化变量来存储模型
final_model = None

# 进行 10 折交叉验证，但只保留最后一次折叠的模型
for fold, (train_index, val_index) in enumerate(kf.split(X_selected)):
    X_train_fold, X_val_fold = X_selected.iloc[train_index], X_selected.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
    # 创建 ElasticNet 模型并进行训练
    model = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio, random_state=42)
    model.fit(X_train_fold, y_train_fold)
    
    # 如果是最后一次折叠，保存模型
    if fold == 9:  # 最后一折，索引为 9
        final_model = model
        # 评估模型
        y_pred_val = model.predict(X_val_fold)
        mse = mean_squared_error(y_val_fold, y_pred_val)
        r2 = r2_score(y_val_fold, y_pred_val)
        print(f"Fold {fold + 1} - MSE: {mse}, R²: {r2}")

# 确保最终模型被训练
if final_model:
    # 在最后一折的训练集上进行评估
    y_pred = final_model.predict(X_val_fold)  # 使用最后一折的验证集进行预测
    final_mse = mean_squared_error(y_val_fold, y_pred)
    final_r2 = r2_score(y_val_fold, y_pred)

    # 输出评估结果
    print(f"最后一次折叠模型的 MSE: {final_mse}")
    print(f"最后一次折叠模型的 R²: {final_r2}")

    # 保存最后一次折叠的模型
    model_file_path = 'D:/adult_dep/12.28HAMD拟合/all_model.joblib'
    joblib.dump(final_model, model_file_path)
    print(f"模型已保存到 {model_file_path}")
else:
    print("没有训练出模型。")


In [ ]:
# 读取特征名称
features_names = pd.read_csv('D:/代谢组/data/features_names.csv', header=None).iloc[0].tolist()

# 筛选特征
X_train_selected = X_train[features_names]
X_test_selected = X_test[features_names]

# 保存数据集
X_train_selected.to_csv('D:/代谢组/data/X_train_selected.csv', index=False)
X_test_selected.to_csv('D:/代谢组/data/X_test_selected.csv', index=False)

In [ ]:
X_train_selected

In [ ]:
# 重复 100 次模型训练和评估
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score


# Bootstrap 重采样和模型评估
bootstrap_iterations = 100
correlations = []
mse_list = []
r2_list = []

for _ in range(bootstrap_iterations):
    bootstrap_indices = np.random.choice(range(X_train_selected.shape[0]), size=int(0.8 * X_train_array.shape[0]), replace=True)
    out_of_bag_indices = np.setdiff1d(range(X_train_selected.shape[0]), bootstrap_indices)
    
    X_train_bootstrap = X_train_selected[bootstrap_indices]
    y_train_bootstrap = y_train_array[bootstrap_indices]
    X_val_oob = X_train_selected[out_of_bag_indices]
    y_val_oob = y_train_array[out_of_bag_indices]
    
    elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=42)
    elastic_net.fit(X_train_bootstrap, y_train_bootstrap)
    
    y_pred_oob = elastic_net.predict(X_val_oob)
    mse = mean_squared_error(y_val_oob, y_pred_oob)
    r2 = r2_score(y_val_oob, y_pred_oob)
    corr = np.corrcoef(y_val_oob, y_pred_oob)[0, 1]
    
    mse_list.append(mse)
    r2_list.append(r2)
    correlations.append(corr)

# 提取相关性值
correlation_values = correlations

# 绘制 Pearson 相关系数分布直方图
plt.figure(figsize=(8, 6))
sns.histplot(correlation_values, bins=25, kde=False)
plt.xlabel('Correlation with age in testing set')
plt.ylabel('Frequency')
plt.title('Distribution of Pearson Correlation Coefficients')
pdf_file_path = 'D:/代谢组/data/correlation_distribution.pdf'
plt.savefig(pdf_file_path, format='pdf')
plt.show()

# 计算 MSE 和 R² 的平均值
avg_mse = np.mean(mse_list)
avg_r2 = np.mean(r2_list)

print(f"平均 MSE: {avg_mse}")
print(f"平均 R²: {avg_r2}")

In [ ]:
# 构建最终模型（步骤3）
import joblib

final_model = ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=42)

final_model.fit(X_train_selected, y_train)

# 保存模型
model_file_path = 'D:/代谢组/data/elastic_net_model.joblib'
joblib.dump(final_model, model_file_path)
print(f"模型已保存到 {model_file_path}")

In [ ]:
import joblib

# 加载模型
model_file_path = 'D:/代谢组/data/elastic_net_model.joblib'
final_model = joblib.load(model_file_path)

X_test_selected = pd.read_csv('D:/代谢组/data/X_test_selected.csv')
# 使用加载的模型进行预测
y_pred_test = final_model.predict(X_test_selected)

In [ ]:
X_test_selected

In [ ]:
y_pred_test

In [ ]:

# 绘制代谢组学年龄与实际年龄的关系图
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr


# 加载模型
model_file_path = 'D:/代谢组/data/elastic_net_model.joblib'
loaded_model = joblib.load(model_file_path)
print(f"模型已从 {model_file_path} 加载")

final_model.fit(X_train_selected, y_train)

# 使用训练好的模型对测试集进行预测
y_pred_test = final_model.predict(X_test_selected)

# 计算 Pearson 相关系数和平均绝对误差 (MAE)
pearson_corr, _ = pearsonr(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)

print(f"Pearson相关系数: {pearson_corr}")
print(f"平均绝对误差 (MAE): {mae}")

# 绘制散点图
# plt.figure(figsize=(8, 6))
# sns.scatterplot(x=y_test, y=y_pred_test, alpha=0.7)
# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
# plt.xlabel('Chronological Age')
# plt.ylabel('Metabolomic Age')
# plt.title('Relationship between Chronological Age and Metabolomic Age')
# plt.text(x=0.05, y=0.95, s=f'Pearson: {pearson_corr:.2f}\nMAE: {mae:.2f}\nR²: {r2:.2f}', 
#          ha='left', va='top', transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
# plt.grid(True)
# pdf_file_path = 'D:/代谢组/data/线性关系图.pdf'
# plt.savefig(pdf_file_path, format='pdf')
# plt.show()



# # 绘制散点图
# plt.figure(figsize=(8, 6))
# sns.scatterplot(x=y_test, y=y_pred_test, alpha=0.6, edgecolor=None)
# sns.regplot(x=y_test, y=y_pred_test, scatter=False, color='black')
# plt.xlabel('Chronological Age')
# plt.ylabel('Metabolomic Age')
# plt.title('Relationship between Chronological Age and Metabolomic Age')
# pdf_file_path = 'D:/代谢组/data/线性关系图2.pdf'
# plt.savefig(pdf_file_path, format='pdf')
# plt.show()



# 绘制散点图
plt.figure(figsize=(8, 8))  # 设置图形为正方形
sns.scatterplot(x=y_test, y=y_pred_test, alpha=0.6, edgecolor=None)
sns.regplot(x=y_test, y=y_pred_test, scatter=False, color='black')
# 设置坐标轴范围
plt.xlim(20, 65)
plt.ylim(20, 65)
# 设置坐标轴标签和标题
plt.xlabel('Chronological Age')
plt.ylabel('Metabolomic Age')
plt.title('Relationship between Chronological Age and Metabolomic Age')
# 保存图形为 PDF 文件
pdf_file_path = 'D:/代谢组/data/线性关系图_new.pdf'
plt.savefig(pdf_file_path, format='pdf')
# 显示图形
plt.show()

In [ ]:
# 计算生物学年龄和生物学年龄加速（mAA）
# y_pred_test = final_model.predict(X_test_selected)
mAA = y_pred_test - y_test

# 绘制生物学年龄加速的分布直方图
plt.figure(figsize=(8, 6))
sns.histplot(mAA, kde=True)
plt.xlabel('Metabolomic Age Acceleration (mAA)')
plt.ylabel('Frequency')
plt.title('Distribution of Metabolomic Age Acceleration')
pdf_file_path = 'D:/代谢组/data/mAA_result.pdf'
plt.savefig(pdf_file_path, format='pdf')
plt.show()

# 保存生物学年龄和生物学年龄加速数据
df_mAA = pd.DataFrame({
    'Chronological Age': y_test,
    'Metabolomic Age': y_pred_test,
    'Metabolomic Age Acceleration (mAA)': mAA
})

csv_file_path_mAA = 'D:/代谢组/data/mAA_results.csv'
df_mAA.to_csv(csv_file_path_mAA, index=False)
print(f"生物学年龄和生物学年龄加速结果已保存到 {csv_file_path_mAA}")